<a href="https://colab.research.google.com/github/LindsayTeeters/Challenge-16-_Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 https://cloud.r-project.org/bin/li

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles

spark = SparkSession.builder.appName("Challenge16_AmazonVineAnalysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...|2015-08-31 00:00:00|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...| 

In [8]:
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REAKC26P07MDN|          5|            0|          0|   N|                Y|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|                Y|
|R14QJW3XF8QO1P|          5|            0|          0|   N|                Y|
|R2HB7AX0394ZGY|          5|            0|          0|   N|                Y|
| RGKMPDQGSAHR3|          5|            0|          0|   N|                Y|
|R1DJCVPQGCV66E|          5|            0|          0|   N|                Y|
|R3V52EAWLPBFQG|          3|            0|          0|   N|                Y|
|R3DKO8J1J28QBI|          2|            0|          0|   N|                Y|
| R764DBXGRNECG|          5|            1|          1|   N|                N|
| RW1853GAT0Z9F|          5|            0|          0|   N|     

In [9]:
vine_votes_df = vine_df.filter(vine_df.total_votes >=20)
vine_votes_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|
|R3A1VQ3SQDXEJ3|          5|           20|         20|   N|                Y|
|  R39GSNW76GYF|          1|           20|         23|   N|                Y|
| RPJLR6MFDPXXE|          5|           35|         36|   N|     

In [10]:
vine_vote_filter = vine_votes_df.filter(vine_votes_df.helpful_votes/vine_votes_df.total_votes >= 0.5)
vine_vote_filter.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|
|R3A1VQ3SQDXEJ3|          5|           20|         20|   N|                Y|
|  R39GSNW76GYF|          1|           20|         23|   N|                Y|
| RPJLR6MFDPXXE|          5|           35|         36|   N|     

In [11]:
vine_review_df = vine_vote_filter.filter(vine_vote_filter.vine == 'Y')
vine_review_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3A71VR1JZD8WF|          2|           27|         30|   Y|                N|
|R16OMUJIGI18JZ|          5|           72|         72|   Y|                N|
|R3TS8ZP2FHQ9XR|          5|           39|         42|   Y|                N|
|R2MHP919VZN7DI|          5|           29|         30|   Y|                N|
| RD2BCTVS59A5L|          2|           20|         20|   Y|                N|
|R1JUJLXQ2CMWKF|          4|           25|         26|   Y|                N|
|R2T7YE0IFI6N9L|          3|           24|         25|   Y|                N|
|R25FI3J7WWOYXM|          3|           55|         58|   Y|                N|
|R3P3U4EOWIUD50|          3|           27|         36|   Y|                N|
|R3C7MKX7MO6XUC|          5|          216|        219|   Y|     

In [14]:
vine_review_no = vine_vote_filter.filter(vine_vote_filter.vine == 'N')
vine_review_no.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|
|R3A1VQ3SQDXEJ3|          5|           20|         20|   N|                Y|
|  R39GSNW76GYF|          1|           20|         23|   N|                Y|
| RPJLR6MFDPXXE|          5|           35|         36|   N|     

In [15]:
vine_review_df.summary().show()


+-------+--------------+------------------+------------------+------------------+----+-----------------+
|summary|     review_id|       star_rating|     helpful_votes|       total_votes|vine|verified_purchase|
+-------+--------------+------------------+------------------+------------------+----+-----------------+
|  count|           170|               170|               170|               170| 170|              170|
|   mean|          null| 3.929411764705882|63.641176470588235| 68.03529411764706|null|             null|
| stddev|          null|1.1124708391270806|107.53304464635703|110.81862649267784|null|             null|
|    min|R10A5XNAP2Y4N7|                 1|                13|                20|   Y|                N|
|    25%|          null|                 3|                25|                27|null|             null|
|    50%|          null|                 4|                35|                38|null|             null|
|    75%|          null|                 5|            

In [17]:
vine_5star = vine_review_df.filter(vine_review_df.star_rating == 5)
vine_5star.summary().show()


+-------+--------------+-----------+-----------------+-----------------+----+-----------------+
|summary|     review_id|star_rating|    helpful_votes|      total_votes|vine|verified_purchase|
+-------+--------------+-----------+-----------------+-----------------+----+-----------------+
|  count|            65|         65|               65|               65|  65|               65|
|   mean|          null|        5.0|57.52307692307692|61.33846153846154|null|             null|
| stddev|          null|        0.0|63.17018375297491|65.84837624304912|null|             null|
|    min|R10A5XNAP2Y4N7|          5|               15|               20|   Y|                N|
|    25%|          null|          5|               26|               28|null|             null|
|    50%|          null|          5|               35|               38|null|             null|
|    75%|          null|          5|               57|               62|null|             null|
|    max| RV5OQXGCYHSC6|          5|    

In [18]:
vine_review_no.summary().show()


+-------+--------------+------------------+------------------+------------------+-----+-----------------+
|summary|     review_id|       star_rating|     helpful_votes|       total_votes| vine|verified_purchase|
+-------+--------------+------------------+------------------+------------------+-----+-----------------+
|  count|         37840|             37840|             37840|             37840|37840|            37840|
|   mean|          null|3.7647463002114163| 57.50052854122622|61.863583509513745| null|             null|
| stddev|          null|1.6055561346315699|107.60541722886974|111.92734632290404| null|             null|
|    min|R10037MJ9LKCVM|                 1|                10|                20|    N|                N|
|    25%|          null|                 2|                23|                25| null|             null|
|    50%|          null|                 5|                31|                34| null|             null|
|    75%|          null|                 5|   

In [19]:
non_vine_5star = vine_review_no.filter(vine_review_no.star_rating == 5)
non_vine_5star.summary().show()


+-------+--------------+-----------+-----------------+------------------+-----+-----------------+
|summary|     review_id|star_rating|    helpful_votes|       total_votes| vine|verified_purchase|
+-------+--------------+-----------+-----------------+------------------+-----+-----------------+
|  count|         20612|      20612|            20612|             20612|20612|            20612|
|   mean|          null|        5.0|59.70216378808461|63.142198719192706| null|             null|
| stddev|          null|        0.0|99.94106277939113|102.87597068559373| null|             null|
|    min|R10037MJ9LKCVM|          5|               11|                20|    N|                N|
|    25%|          null|          5|               23|                25| null|             null|
|    50%|          null|          5|               33|                35| null|             null|
|    75%|          null|          5|               56|                60| null|             null|
|    max| RZZMYQ9G22

In [20]:
verified_purchase_df = vine_df.filter(vine_df.verified_purchase == 'Y')
verified_purchase_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REAKC26P07MDN|          5|            0|          0|   N|                Y|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|                Y|
|R14QJW3XF8QO1P|          5|            0|          0|   N|                Y|
|R2HB7AX0394ZGY|          5|            0|          0|   N|                Y|
| RGKMPDQGSAHR3|          5|            0|          0|   N|                Y|
|R1DJCVPQGCV66E|          5|            0|          0|   N|                Y|
|R3V52EAWLPBFQG|          3|            0|          0|   N|                Y|
|R3DKO8J1J28QBI|          2|            0|          0|   N|                Y|
| RW1853GAT0Z9F|          5|            0|          0|   N|                Y|
|R33GITXNUF1AD4|          2|            0|          0|   N|     

In [21]:
non_verified_purchase_df = vine_df.filter(vine_df.verified_purchase == 'N')
non_verified_purchase_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R764DBXGRNECG|          5|            1|          1|   N|                N|
|R1II0M01NIG293|          2|            0|          0|   N|                N|
|R29RDHIIKHRIWR|          5|            0|          0|   N|                N|
| RFL3O67OLCAOY|          5|            0|          0|   N|                N|
|R33YBTQ4PDRB4X|          5|            0|          0|   N|                N|
|R2Q0XD0PAJWYBX|          1|            0|          0|   N|                N|
| RHLK7LWWF74A0|          1|            0|          0|   N|                N|
| RGVPPJ3PGHLBW|          5|            0|          0|   N|                N|
|R2JBRNRGK9213U|          5|            0|          0|   N|                N|
|R2I6ZY7OANUVN9|          1|            1|          1|   N|     

In [25]:
non_verified_purchase = vine_df.filter(vine_df.verified_purchase == 'N')
non_verified_purchase.summary().show()

+-------+--------------+------------------+------------------+-----------------+------+-----------------+
|summary|     review_id|       star_rating|     helpful_votes|      total_votes|  vine|verified_purchase|
+-------+--------------+------------------+------------------+-----------------+------+-----------------+
|  count|        320076|            320076|            320076|           320076|320076|           320076|
|   mean|          null|3.8635074169884653|3.7105937339881776|4.357993101638361|  null|             null|
| stddev|          null| 1.545842902919736| 25.76292358345367| 27.3686983578364|  null|             null|
|    min|R1000JOVLD0J41|                 1|                 0|                0|     N|                N|
|    25%|          null|                 3|                 0|                0|  null|             null|
|    50%|          null|                 5|                 0|                1|  null|             null|
|    75%|          null|                 5|   

In [26]:
verified_purchase = vine_df.filter(vine_df.verified_purchase == 'Y')
verified_purchase.summary().show()

+-------+--------------+------------------+------------------+------------------+-------+-----------------+
|summary|     review_id|       star_rating|     helpful_votes|       total_votes|   vine|verified_purchase|
+-------+--------------+------------------+------------------+------------------+-------+-----------------+
|  count|       2323538|           2323538|           2323538|           2323538|2323538|          2323538|
|   mean|          null| 4.182244060566257|1.3694275712297368|1.6055403440787281|   null|             null|
| stddev|          null|1.2887472323692106|12.405541982711743|12.943184426933449|   null|             null|
|    min|R100065P6TTS3J|                 1|                 0|                 0|      N|                Y|
|    25%|          null|                 4|                 0|                 0|   null|             null|
|    50%|          null|                 5|                 0|                 0|   null|             null|
|    75%|          null|    